# Домашнее задание к вебинару 9

Задание: обучить генератор текстов
Обучаем генератор символов (будет предсказывать следующий символ)

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
path_to_file = '/content/evgenyi_onegin.txt'

In [6]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [7]:
# Выведем фрагмент текста
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [8]:
# Вывелем все уникальные символы (буквы+символы) в тексте
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [9]:
vocab[:40]

['\n',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S']

In [10]:
# Задаем отображения из множества уникальных букв в множество индексов
char2idx = {u:i for i, u in enumerate(vocab)} # словарь {индекс: символ (буква или символ)}

idx2char = np.array(vocab) # array из символов словаря (можно по индексу извлекать букву)

In [11]:
idx2char

array(['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2',
       '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C',
       'D', 'E', 'F', 'G', 'H', 'I', 'L', 'M', 'N', 'O', 'P', 'Q', 'R',
       'S', 'T', 'V', 'W', 'X', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g',
       'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
       'v', 'w', 'y', 'z', '{', '}', 'А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж',
       'З', 'И', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф',
       'Х', 'Ц', 'Ч', 'Ш', 'Ь', 'Э', 'Ю', 'Я', 'а', 'б', 'в', 'г', 'д',
       'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с',
       'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю',
       'я'], dtype='<U1')

In [12]:
# Теперь переводим все символы текста в индексы
text_as_int = np.array([char2idx[c] for c in text])

In [13]:
text_as_int

array([ 71, 110, 104, ..., 104, 121,   0])

In [16]:
print(text[:500]), print(text_as_int[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо
[ 71 110 104 109 116  99 112 103 115   1  87 104 115 102 104 104 101 107
 122   1  85 118 123 109 107 112   0   0   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1  76 101 102 104 112 107 108   1  84 112 104 102
 107 112   0   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1  86
 113 111  99 112   1 101   1 116 117 107 120  99 120   0   0 

(None, None)

### Подготовка данных к обучению

In [67]:
# Максимальная длина предложения (максимальное количество слов в батче) для входных данных (в буквах)
seq_length = 100

# Количество эпох
examples_per_epoch = len(text)//(seq_length+1)

# создаем датасет из данных
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()], len(char_dataset)) 

А 286984
л 286984
е 286984
к 286984
с 286984


In [68]:
# Создаем последовательность батчей (размер батча = 101)
sequences = char_dataset.batch(seq_length+1, drop_remainder = True)

In [69]:
for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [70]:
# Разбиваем каждый батч на признаки и целевую переменную (следующую букву)
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

#применяем функцию split_input_target ко всем батчам
dataset = sequences.map(split_input_target)

In [71]:
for input_example, target_example in dataset:
    print(f'Количество батчей: {len(dataset)}')
    print(f'Данные: {input_example}')
    print(f'Целевая переменная: {target_example}')
    break

Количество батчей: 2841
Данные: [ 71 110 104 109 116  99 112 103 115   1  87 104 115 102 104 104 101 107
 122   1  85 118 123 109 107 112   0   0   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1  76 101 102 104 112 107 108   1  84 112 104 102
 107 112   0   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1]
Целевая переменная: [110 104 109 116  99 112 103 115   1  87 104 115 102 104 104 101 107 122
   1  85 118 123 109 107 112   0   0   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1  76 101 102 104 112 107 108   1  84 112 104 102 107
 112   0   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1]


In [72]:
for input_example, target_example in dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


### Построение модели

In [73]:
# размер батча
BATCH_SIZE = 64

# размер буфера для перемешивания данных
BUFFER_SIZE = 10000

# перемешивание разделенных данных
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [74]:
for input_example, target_example in dataset:
    print(input_example.shape, target_example.shape)

(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)
(64, 100) (64, 100)


In [75]:
# Длина словаря
vocab_size = len(vocab)

# Длина выходного эмбеддинга
embedding_dim = 256

# Количество скрытых состояний в RNN слое 
rnn_units = 1024

In [88]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [89]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [90]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (64, None, 256)           33536     
                                                                 
 lstm_11 (LSTM)              (64, None, 1024)          5246976   
                                                                 
 lstm_12 (LSTM)              (64, None, 1024)          8392704   
                                                                 
 lstm_13 (LSTM)              (64, None, 1024)          8392704   
                                                                 
 lstm_14 (LSTM)              (64, None, 1024)          8392704   
                                                                 
 dense_3 (Dense)             (64, None, 131)           134275    
                                                                 
Total params: 30,592,899
Trainable params: 30,592,899


**Сделаем предсказание без обучения модели**

In [91]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape," # (batch_size, sequence_length, vocab_size)")

(64, 100, 131)  # (batch_size, sequence_length, vocab_size)


In [92]:
len(dataset)

44

In [93]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 131), dtype=float32, numpy=
array([[-1.81417954e-06,  4.00193403e-06,  3.88547869e-06, ...,
        -1.64471712e-05,  1.55858397e-05, -1.20994455e-05],
       [-2.22608378e-05,  1.18116441e-06, -2.58475138e-05, ...,
        -2.25488366e-05,  5.01978466e-05, -3.32598174e-05],
       [-5.70883458e-05, -3.95189818e-05, -9.98164469e-05, ...,
        -4.09461973e-06,  9.66930165e-05, -5.00300594e-05],
       ...,
       [-3.15367640e-03, -1.52308925e-03, -6.85384206e-04, ...,
         1.61391473e-03, -1.53077301e-03, -9.19225393e-04],
       [-3.02344514e-03, -1.38021854e-03, -7.05452985e-04, ...,
         1.56130770e-03, -1.39190443e-03, -8.37130705e-04],
       [-2.87503889e-03, -1.32162811e-03, -8.16532178e-04, ...,
         1.46284408e-03, -1.28053129e-03, -7.66042853e-04]], dtype=float32)>

In [94]:
# сэмплируем предсказание 
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1) #количество независимых выборок 1

#убираем лишнюю размерность (список индексов)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([ 15,   4,  68,  61,  70,  43,   6,  54,  64, 126, 114,  94,  31,
        96, 104,  68, 119,  68,  68, 102,  39,  20,  93,  94,  20,   0,
       128,   6,   2,  87, 114,  91,  91,  36,  83,   3,  47, 101,  54,
        10,  11,  99,  19, 111,  59,   6,  11,  41,   0,  56,  11,  80,
       126,  55,  82,  11,  74,  27, 122,  25,  80,  18, 117,   4, 126,
        23,  18,  25, 104,  63,  16, 101,   1,  93,   1,  91, 109,   7,
       111,  75, 120,  24,  12, 124, 111,  15,  84,  52,  24,  18,  91,
        18, 125, 107, 126,  65,  24,   3,  89,   4])

In [95]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ц,\n                        Давал три бала ежегодно\n                        И промотался наконец.\n   '

Next Char Predictions: 
 '5\'zr}X)kuыпШIЭеzфzzгS:ЧШ:\nэ)!СпХХPН"cвk01а9мp)1V\nm1КыlМ1ГEчCК8т\'ыA8Cеt6в Ч Хк,мДхB2щм5ОhB8Х8ъиыvB"У\''


In [96]:
# функция потерь

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

# подсчитаем ошибку
example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 131)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.876249


Обучение модели

In [97]:
# место для хранения checkpoint
checkpoint_dir = '/content/training_checkpoints'
# Имя файла checkpoint
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=44*50, # сохраняем каждую 50-ю итерацию - в эпохе 44 батча (это видно при запуске обучения)
    save_weights_only=True, # будем сохранять только веса
    )

In [98]:
# Компиляция модели
model.compile(optimizer='adam', loss=loss)

In [99]:
#обучение модели
EPOCHS = 100
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
44/44 [==============================] - 21s 346ms/step - loss: 2.4436
Epoch 2/100
44/44 [==============================] - 16s 355ms/step - loss: 1.9419
Epoch 3/100
44/44 [==============================] - 16s 359ms/step - loss: 1.6919
Epoch 4/100
44/44 [==============================] - 16s 358ms/step - loss: 1.5325
Epoch 5/100
44/44 [==============================] - 16s 361ms/step - loss: 1.4525
Epoch 6/100
44/44 [==============================] - 16s 362ms/step - loss: 1.3954
Epoch 7/100
44/44 [==============================] - 16s 363ms/step - loss: 1.3670
Epoch 8/100
44/44 [==============================] - 16s 365ms/step - loss: 1.3444
Epoch 9/100
44/44 [==============================] - 16s 361ms/step - loss: 1.3255
Epoch 10/100
44/44 [==============================] - 16s 362ms/step - loss: 1.3126
Epoch 11/100
44/44 [==============================] - 16s 362ms/step - loss: 1.2918
Epoch 12/100
44/44 [==============================] - 16s 364ms/step - loss: 1.2773
E

In [100]:
#предсказание модели
example_batch_predictions = model(input_example_batch)
print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [101]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [102]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ц,\n                        Давал три бала ежегодно\n                        И промотался наконец.\n   '

Next Char Predictions: 
 'а\n                        Тивнл яри бала ежегодно\n                        И,воомотался наконец.\n    '


### Генерация текста

In [103]:
#Находим имя файла последней сохраненной контрольной точки
tf.train.latest_checkpoint(checkpoint_dir)

'/content/training_checkpoints/ckpt_100'

In [104]:
#строим модель
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

#загружаем веса из последней сохраненной контрольной точки в модель
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [105]:
#информация о модели
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (1, None, 256)            33536     
                                                                 
 lstm_15 (LSTM)              (1, None, 1024)           5246976   
                                                                 
 lstm_16 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 lstm_17 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 lstm_18 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 dense_4 (Dense)             (1, None, 131)            134275    
                                                                 
Total params: 30,592,899
Trainable params: 30,592,899


In [126]:
def generate_text(model, start_string):
    # Этап оценки (генерация текста с использованием обученной модели)

    # число букв для генераци
    num_generate = 100

    # Преобразование начальной строки в числа (векторизация)
    input_eval = [char2idx[s] for s in start_string]
    #Возвращаем тензор с осью длины 1, вставленной первой в индекс
    input_eval = tf.expand_dims(input_eval, 0)

    # Пустая строка для хранения результатов
    text_generated = []

    # Низкая температура приводит к более предсказуемому тексту.
    # Более высокая температура приводит к более неожиданному тексту.
    temperature = 0.001

    # здесь batch size == 1
    # сбрасываем состояния всех слоев в модели
    model.reset_states()
    for i in range(num_generate):

        #получаем предсказания модели
        predictions = model(input_eval)

        #удаляем первую размерность в предсказании
        predictions = tf.squeeze(predictions, 0)

        # использование категориального распределения для прогнозирования символа, возвращаемого моделью
        predictions = predictions / temperature

        # выберем последний токен из отсэмплированных предсказаний[-1], т.к. именно он будет предсказанным следующим токеном в строке
        # индекс 0 - выбираем именно индекс токена (помимо него выводится еще размер [1]и тип [2])
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        # print(predicted_id) 

        # Передаем предсказанный символ в качестве следующего ввода в модель, по нему будет предсказывать следующий символ
        input_eval = tf.expand_dims([predicted_id], 0)

        #сохраняем предсказанную букву
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))
        

In [128]:
text_ = generate_text(model, start_string=u"Вот он идет ")
print(text_)

Вот он идет был обмотом,
                        По старине торжествовали
                        И в самом ужас
